# Введение, примеры задач, бизнес- и ML-метрики 

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
prices_recommended = [400, 60, 40, 40, 90]
prices_bought = [20, 50, 10, 100]

In [3]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    flags = np.isin(bought_list, recommended_list[:k])
    recall_k = flags.sum()/k
    
    return recall_k


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    #TODO: Ваш код здесь
    flags = np.isin(bought_list, recommended_list[:k])
    money_recall_k = (flags * prices_bought).sum() / prices_recommended[:k].sum()
    
    return money_recall_k

In [4]:
recall(recommended_list, bought_list)

0.5

In [5]:
recall_at_k(recommended_list, bought_list, k=5)

0.4

In [6]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.1746031746031746

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [7]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    if not np.isin(bought_list, recommended_list[:k]).sum():
        return 0
    mrr_k = 1 / (np.argmax(np.isin(bought_list, recommended_list[:k])) + 1)
    return mrr_k

In [8]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
mrr_at_k(recommended_list, bought_list, k=10)

0.3333333333333333

In [9]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [143, 32, 521, 991]
mrr_at_k(recommended_list, bought_list, k=10)

1.0

In [10]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [69, 32, 521, 991]
mrr_at_k(recommended_list, bought_list, k=2)

0

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [11]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    flags = np.isin( recommended_list[:k], bought_list)
    dcg_at_k = sum([int(flags[index]) / np.log2(index + 2) for index in range(k)])
    ideal_dcg_at_k = sum([1 / np.log2(i + 1) for i in range(1, k + 1)])
    ndcg_k = dcg_at_k / ideal_dcg_at_k
    
    return ndcg_k
    

In [12]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
ndcg_at_k(recommended_list, bought_list, k=5)

0.48522855511632257

In [13]:
recommended_list = [143, 521, 32, 143, 991, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
ndcg_at_k(recommended_list, bought_list, k=5)

1.0

In [14]:
recommended_list = [1413, 156, 1134, 9911, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
ndcg_at_k(recommended_list, bought_list, k=5)

0.0